In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy

## PageRank Algorithm on a Simple Graph
Here, I have attempted to implement the pageRank algorithm on a simple hardcoded graph. 
* Used to help me understand the working of the algorithm on a simpler dimension of data and solifying the mathematical foundation of PageRank
  - Markov Process
  - Importance of Damping Factor
  - Graphs
  - Linear Algebra

In [2]:
searchGraph = {
    "1": ["2", '3'],
    "2": ['4'],
    "3": ['7', '6'],
    '4': ['4', '5'],
    '5': ['6'],
    '6': ['6', '3'],
    '7': ['7']
}

In [3]:
n = len(searchGraph)
PR_matrix = np.zeros((n,n))

In [4]:
PR_matrix

array([[0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.]])

In [5]:
for node in searchGraph:
    size = len(searchGraph[node])
    point = int(node)
    relationPoints = set([int(relation)-1 for relation in searchGraph[node]])
    relationArray = PR_matrix[point-1]
    #print(relationPoints)
    for i in range(len(relationArray)):
        if i in relationPoints:
            PR_matrix[point-1, i] = 1/size

In [6]:
linkMatrix = PR_matrix.T
linkMatrix

array([[0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0.5, 0. , 0. , 0. , 0. , 0. , 0. ],
       [0.5, 0. , 0. , 0. , 0. , 0.5, 0. ],
       [0. , 1. , 0. , 0.5, 0. , 0. , 0. ],
       [0. , 0. , 0. , 0.5, 0. , 0. , 0. ],
       [0. , 0. , 0.5, 0. , 1. , 0.5, 0. ],
       [0. , 0. , 0.5, 0. , 0. , 0. , 1. ]])

In [7]:
#damping link matrix
damping_factor = 0.25
linkMatrix = damping_factor * linkMatrix + (1-damping_factor)/len(searchGraph)
linkMatrix = np.round(linkMatrix, 2)

In [8]:
noOfLinks = len(searchGraph)
r = [ 1/noOfLinks for i in range(noOfLinks)]
nextR = linkMatrix * r
zeroVec = np.zeros(noOfLinks)

iter = 0
while iter <= 100 :
    nextR = np.dot(linkMatrix,r)
    if (nextR - r == zeroVec).all():
        break
    r = nextR
    iter += 1

In [9]:
r

array([0.44822527, 0.50126202, 0.58812375, 0.64853088, 0.52496341,
       0.73408757, 0.68722551])

In [10]:
nextR

array([0.44822527, 0.50126202, 0.58812375, 0.64853088, 0.52496341,
       0.73408757, 0.68722551])

In [11]:
iter

101

## PageRank Algorithm on an actual dataset
Here, I have implemented the pageRank algorithm on an actual dataset provided by Stanford on wikipedia pages and its links
* Through this implementation I have understood how to work with cleaned data in Pandas
* I have also understood at an intermediate level how a simple operation such as a Markov Process have a large impact on deciding what the user should be shown for a request
* This project also catered in solidifying my understanding of Linear Algebra and its core concepts and programming concepts such as Numpy and Pandas

In [39]:
import pandas as pd
from dotenv import load_dotenv
import os

# Load environment variables from .env
load_dotenv()

# Access variables
secret_key = os.getenv('DATASET_LOCATION')

In [13]:
dataSet = secret_key
df2 = pd.read_excel(dataSet, engine='odf')

In [14]:
df2

,Start Link,End Link
0,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,Bede
1,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,Columba
2,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,D%C3%A1l_Riata
3,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,Great_Britain
4,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,Ireland
...,...,...
119877,Zulu,South_Africa
119878,Zulu,Swaziland
119879,Zulu,United_Kingdom
119880,Zulu,Zambia


In [15]:
import collections
links = collections.defaultdict(list)
for index, record in df2.iterrows():
    links[record["Start Link"]].append(record["End Link"])

In [16]:
noOfPoints = len(links)

In [17]:
data = {f"{link}":[0.0]*noOfPoints for link in links}

In [18]:
linkDf = pd.DataFrame(data)

In [19]:
linkDfColumns = linkDf.columns.tolist()

In [20]:
linkDf = linkDf.rename(index={i: linkDfColumns[i] for i in range(len(linkDfColumns))})

In [21]:
for link in links:
    for node in links[link]:
        linkDf.at[link, node] = 1/len(links[link])

In [22]:
linkMatrix = linkDf.to_numpy()

In [23]:
M,N = linkMatrix.shape
diff = abs(M-N)

extraRows = np.array([[0]*N]*diff)

In [24]:
lM = np.vstack([linkMatrix, extraRows])

In [25]:
lM = np.nan_to_num(lM, nan=0)

In [26]:
lM = lM.T
ROW, COL = lM.shape

In [27]:
#damping link matrix
damping_factor = 0.5
linkMatrix = damping_factor * lM + (1-damping_factor)/ROW
linkMatrix = np.round(linkMatrix, 5)

In [28]:
linkMatrix

array([[0.00011, 0.00011, 0.00011, ..., 0.00011, 0.00011, 0.00011],
       [0.00011, 0.00011, 0.00011, ..., 0.00011, 0.00011, 0.00011],
       [0.00011, 0.00011, 0.00011, ..., 0.00011, 0.00011, 0.00011],
       ...,
       [0.00011, 0.00011, 0.00011, ..., 0.00011, 0.00011, 0.00011],
       [0.00011, 0.00011, 0.00011, ..., 0.00011, 0.00011, 0.00011],
       [0.00011, 0.00011, 0.00011, ..., 0.00011, 0.00011, 0.00011]])

In [29]:
#Page Rank algorithm
M,N = linkMatrix.shape
r = [ 1/M for i in range(M)]
nextR = linkMatrix * r
zeroVec = np.zeros(M)

iter = 0
while iter <= 1000 :
    nextR = np.dot(linkMatrix,r)
    if (nextR - r == zeroVec).all():
        break
    r = nextR
    iter += 1

In [30]:
probsCols = linkDf.columns.tolist()

In [31]:
probs = pd.Series(r, name='Relative Probability')

In [32]:
probs = probs.rename(index={i: probsCols[i] for i in range(len(probsCols))})

In [33]:
ranked_data = probs.sort_values(ascending=False)

In [34]:
ranked_data = pd.DataFrame(ranked_data)

In [35]:
ranked_data = ranked_data.reset_index()

In [36]:
ranked_data.columns = ['Source', ranked_data.columns[-1]]

In [37]:
ranked_data

,Source,Relative Probability
0,United_States,0.791757
1,United_Kingdom,0.467106
2,Europe,0.451464
3,France,0.442236
4,England,0.380710
...,...,...
4587,Hiroh_Kikai,0.012557
4588,History_of_Burnside,0.012557
4589,Raising_the_Flag_on_Iwo_Jima,0.012557
4590,Sydney_Riot_of_1879,0.012557


In [38]:
ranked_data[ranked_data['Source'] == 'California']

,Source,Relative Probability
52,California,0.134917
